# RL Exercise 3 - Asynchronous Advantage Actor Critic

**GOAL:** The goal of this exercise is to demonstrate how to use the asynchronous advantage actor critic (A3C) algorithm.

A3C is described in detail in https://arxiv.org/abs/1602.01783.

TODO: Describe A3C more.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from ray.rllib.a3c import A3C, DEFAULT_CONFIG

Start up Ray. This must be done before we instantiate any RL algorithms. We pass in `num_workers=0` because the training algorithm's constructor will create a number of actors.

In [ ]:
ray.init(num_workers=0)

Instantiate an A3C algorithm object. We pass in a config object that specifies how the network and training procedure should be configured. Some of the parameters are the following.

- `num_workers` is the number of actors that the algorithm will create. This determines the degree of parallelism that will be used.
- `num_batches_per_iterations` is TODO
- `batch_size` is TODO

In [ ]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 3
config['num_batches_per_iteration'] = 100
config['batch_size'] = 10

algorithm = A3C('CartPole-v0', config)

**EXERCISE:** Train the algorithm for some number of steps on the CartPole environment. Compare the performance to PPO from the previous exercise.

In [ ]:
raise Exception('Implement this.')

**EXERCISE:** Instantiate an A3C algorithm object on the `MountainCar-v0` environment and train it for some number of steps. Compare the performance to PPO from the previous exercise.

In [ ]:
raise Exception('Implement this.')